In [ ]:
!python --version


In [ ]:
# TODO: make sure the ps3 is is in my global memeory.
# TODO:  make sure room_id and baord_id are in the global params
# TODO: Make sure the viewport is also passed and in the global memory
# TODO: Ask Luc why this works when creating a stickie (state = {'col': 'red', 'font': '45', 'tex': 'yo'})
# TODO: Tell Luc about no owner when stickies are created programmatically



In [ ]:
from pydantic_ai import Agent, RunContext


In [ ]:
import json
from uuid import UUID
import nest_asyncio
nest_asyncio.apply()

In [ ]:
room_id, board_id = "e07aa346-37dd-4bf5-90bf-fc59a1c3b2e9/d12ec467-6f82-49e0-873a-e15df43e52c4".split("/")
room_id, board_id

In [ ]:
cd ../


In [ ]:

import uuid

from foresight.config import config as conf, prod_type

from foresight.Sage3Sugar.pysage3 import PySage3
ps3 = PySage3(conf, prod_type)

In [ ]:
ps3.s3_comm.get_users()

In [ ]:
b = ps3.rooms[room_id].boards[board_id]
list(b.smartbits)


In [ ]:
sb = list(b.smartbits)
sb

In [ ]:
# sb = b.smartbits[UUID('58c42c91-2a70-426d-a43f-0285901358b7')]
# sb.state.text = "Hi There"
# sb.send_updates()

In [ ]:
ps3.get_smartbits_by_type("stickie", room_id, board_id)

In [ ]:
### Creating Stuff

state = {"text": "Python said Yolo", 'color': 'orange', 'fontSize': '45'}
data = {'size': {'width': 400, 'height': 200, 'depth': 0}, 'position': {'x':1501140.0, "y":1500150.0+100, 'z':0.0}}
response = ps3.create_app(room_id, board_id, 'Stickie', state, data)


In [ ]:
ps3.get_smartbits_by_type("stickie", room_id, board_id)

In [ ]:
state = {"webviewurl": "https://sage-3.github.io/"}
data = {'size': {'width': 500, 'height': 500, 'depth': 0}, 'position': {'x':1501618.0491539645, 'y':1500355.010865956, 'z':0.0}}
res = ps3.create_app(room_id, board_id, 'Webview', state, data)

In [ ]:
res

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal, List, Union
from enum import Enum

from typing import List
from pydantic import BaseModel, HttpUrl, Field
from typing import Dict
import time
from typing import Dict, List


from enum import Enum

class StickieColor(str, Enum):
    """
    Predefined stickie colors
    """
    RED = "red"
    GREEN = "green"
    BLUE = "blue"
    YELLOW = "yellow"

class StickieState(BaseModel):
    "The data that makes up a stickie"
    text: str = Field(..., description="The text that stick contains")
    color: StickieColor = Field(default=StickieColor.YELLOW, description="The color of the stickie to create")
    fontsize: str = Field(..., description="str representation of text size. Use 45 if not provided")

# class WebViewState(BaseModel):
#     """The data that is required to create a webview, this is what is used ot open
#     webview, browser or  a webpage"""
#     webviewurl: HttpUrl = Field(default="https://google.com")

class Position(BaseModel):
    """
    The position of the app on the board
    """

    x: float = Field(..., description="The x position of the app")
    y: float = Field(..., description="The y position of the app")
    z: float = Field(default=0.0, description="The z position of the app")


class Size(BaseModel):
    """
    The dimensions of the app to create
    """

    width: float = Field(default=200, description="The width of the app")
    height: float = Field(default=200, description="The height of the app")
    depth: float = Field(default=0, description="The depth of the app")        
        
class Data(BaseModel):
    size: Size
    position: Position    
        

class SmarbitType(str, Enum):
    """
    Available smartbit types in SAGE3
    """
    STICKIE = "Stickie"
    # WEBVIEW = "Webview" 
        
class SmarbitApp(BaseModel):
    "Defines the SmartbitApp type, which is software application that exists on specialized desktop"

    smartbit_type: SmarbitType
    state: Union[StickieState]
    data: Data



class AppId(BaseModel):
    """The app_id serving as the unique identified of an app on the wall"""
    app_id: UUID
    

sage_smarbits_description = """You are an AI assistant that open application (we sometime also refer to them as "smartbits" or apps) for the SAGE3 operating system. 
Users interact with SAGE3 through a display wall or board; this is similar to a desktop for an OS."""


# class WallAIMessage(BaseModel):
#     "The message for the AI regarding a query"
#     message: str

        
# class WallAIMessage(BaseModel):
#     reponse: Union[WallAIMessage, SmarbitApp]                
        

In [3]:
import os
from getpass import getpass

if "ANTHROPIC_API_KEY" not in os.environ:
    os.environ["ANTHROPIC_API_KEY"] = getpass("Enter your Anthropic API key: ")



In [4]:
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your openAI API key: ")


Enter your openAI API key:  ········


In [ ]:
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass("Enter your openAI API key: ")


In [ ]:
#!pip install pydantic-ai

In [ ]:

# from pydantic_ai.messages import ()

In [ ]:




prompt_create_app = f"""{sage_smarbits_description}
Your job is to listen to users' natural language requests and translate them into the precise JSON parameters needed to 
create (may refer to it as opne) or remove(may refer to it as delete or close) apps  on the SAGE3 wall. 
When users describe what they want to create, you'll convert their request into the proper technical format that SAGE3 requires to call the app. 
You understand all SAGE3 smartbit capabilities and will recommend the best app type for each user's needs.

To find a suitable location to create an app, please use the get_available_space function.
To create an app, use create_apps function, passing it the list of apps that need ot be created. 




"""

create_delete_app_agent = Agent(  
    "claude-3-5-haiku-latest",
    result_type=SmarbitApp,    
    system_prompt=prompt_create_app,
     model_settings={'temperature': 0.0}    
)
@create_delete_app_agent.tool_plain 
def get_available_space(nb_apps: int) -> Position:
    """  Returns a suitable X, Y, Z location where each app can be created.
    
    This position should be used in the Data object of the SmarbitApp being created.
    nb_apps: int is the number of apps we need to get space for. be positioned
    """
    
    return [Position(**{'x': 1501973.615403275, 'y': 1500349.2037941106, 'z': 0})] * nb_apps


@create_delete_app_agent.tool_plain 
def create_apps(sb_list: List[SmarbitApp]) -> str:
    """ 
        This function creates one or more smartbit application given a list of SmarbitApp data to initialize the Apps. 
        sb_list: List[SmarbitApp] list of object of type SmarbitApp, where each object provides the type of the smartbit, it's data and it's state (what it holds)
    
    """
    print(f"function create_apps called withg {sb_list}")
    returned_messages = []
    for sb in sb_list:
        sb_type = sb.smartbit_type
        state = json.loads(sb.state.model_dump_json())
        data = json.loads(sb.data.model_dump_json())
    
        response = ps3.create_app(room_id, board_id, sb_type, state, data)
        if response and type(response.json()) is dict and response.json().get('success', None):
            returned_messages.append(f"App of type {sb_type} with  state:{state} created successfully")
        else:
            # TODO:fix in ps3.create_app 
            returned_messages.append(f"Couldn not create app of type {sb_type} with state:{state}. Error: {response.json()['error']}")
    return returned_messages



In [ ]:
result = await create_delete_app_agent.run("create Two stickies, with popular pizza sauce names")

In [ ]:
users= ps3.s3_comm.get_users()
users

In [ ]:

sage_smarbits_description =  """The SAGE3 operating system lets users interact with applications (called "smartbits") through a display wall or board."""

prompt_select_apps = f"""
{sage_smarbits_description} As an AI assistant, your role is to help users find the exact application(s) they need by intepreting their natural language requests and returning only the app_ids that precisely match their requirements.

When users describe what they're looking for, you need to ensure each suggested application id retured meets their specific criteria.

You can use any of the aviable tools to interact with the wall, include to list currently available apps.

Some selection operations may require multiple steps, so you need to think step by step. 

If none of the apps matches the exact requirements specified in the user query, then return an empty list.
"""


select_app_agent = Agent(  
    "openai:gpt-4o",
    result_type=List[AppId],    
    system_prompt=prompt_select_apps,
     model_settings={'temperature': 0.6}    
)

@select_app_agent.tool_plain
def get_apps_for_context() -> Dict:
    """Returns a dictionary of apps currently on the SAGE3 wall.
    
    Returns a dictionary where keys are app_ids and values are app details.
    App details include information such as position, color, creator, and other
    relevant metadata.
    """
    
    users= ps3.s3_comm.get_users()
    apps = ps3.get_apps(room_id, board_id)
    new_fromatted_apps = {}
    for k, v in apps.items():
        try:
            SmarbitType(v['data']['type'])
            del v['_id']
            del v['_updatedBy']
            del v['_updatedAt']
            del v['data']["rotation"]
            del v['data']["raised"]
            del v['data']["pinned"]
            del v['data']["dragging"]

            if "sources" in v['data']['state']:
                del v['data']['state']["sources"]

            if "lock" in v['data']['state']:
                del v['data']['state']["lock"]
            
            if "executeInfo"  in v['data']['state']:
                del v['data']['state']["executeInfo"]        
        
            user = users.get(v['_createdBy'], ("Unknown", ))
            v['_createdBy'] = user[0]        
            v['_createdAt'] = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(v['_createdAt']// 1000))                
            new_fromatted_apps[k] = v
        except Exception as e:
            print(f"Ommitted {v['data']['type']}")
        
    return new_fromatted_apps




In [ ]:
 get_apps_for_context()

In [ ]:
results = await select_app_agent.run("What stickies are relevant to Moroccan food")
results.data

In [ ]:
b.smartbits[UUID('2a8cbd8e-9905-435f-ac78-8aa0d876093d')]

In [ ]:
results = await select_app_agent.run("Which apps were created by Mahdi")
results.data

In [ ]:
type(ps3.get_apps(room_id, board_id))

In [ ]:
get_app_details([UUID('00592a68-2700-4b69-abb4-340f631fa295'), UUID('e907cca5-e928-441f-8dc6-1343d27e3a44')])

In [ ]:
AppId(app_id = UUID('00592a68-2700-4b69-abb4-340f631fa295'))  == AppId(app_id = UUID('00592a68-2700-4b69-abb4-340f631fa295'))

In [ ]:
#results  = await select_app_agent.run("What stickies are PURPLE")
results.data

In [ ]:
a = []
type(a) is list

In [ ]:

# Assuming sage_smarbits_description is defined somewhere
destructive_operations_prompt = f"""
{sage_smarbits_description}.  

When deleting or moving SAGE3 wall applications (smartbits), first display their app_ids and identifying details. Only proceed after receiving explicit user confirmation. If confirmation is not given, cancel the operation and notify the user that no changes will occur. Use available wall interaction tools to retrieve application details as needed.
If user cancels, be VERY brief and concise and do NOT offer to help the user with something else after.
Use select_apps_to_delete to rreturn the list of AppId to remve
You can use get_app_details to get the details of an app, which you need to show the user for confirmation

Use delete_apps to delete the app once use has confirmed  deletion

"""

destructive_operations_agent = Agent(  
    "claude-3-5-haiku-latest",
    result_type=str,    
    system_prompt=destructive_operations_prompt,
    model_settings={'temperature': 0.5}    
)

@destructive_operations_agent.tool_plain
async def select_apps_to_delete(query_filter: str) -> List[AppId]:
    """
    select the apps to delete based on the condition of the filter that is passed by the user
    query_filter: user query string provided by the  user defining the condition to use to grab the apps the user wants to delete     
    """
    print(f"query_filter is {query_filter}")
    result = await select_app_agent.run(query_filter)
    if result and type(result.data) is not list:
        print(f"Error executing query to select apps to delete {result.data}")
        return None
    elif len(result.data) > 0 and type(result.data[0]) is not AppId:
        print(f"Error returning the select app as an AppId")
        return None
    return result.data
        
@destructive_operations_agent.tool_plain
def get_app_details(app_ids: List[AppId]) -> Dict:
    """
    Returns The detials spefic to the apps whose ids are provided in app_ids
    app_ids: List[AppId] is a list of UUIDs representing the id of each app for which we need to extract the informaiton
    """
    print(f"get_app_details called with {app_ids}")
    all__apps = ps3.get_apps(room_id, board_id)
    return   {k:v for k,v in ps3.get_apps(room_id, board_id).items() if AppId(app_id = UUID(k)) in app_ids}
    
@destructive_operations_agent.tool_plain
def delete_apps(app_ids_to_remove: List[AppId]) -> bool:
    """
    This function takes a list of ids to remove and returns True if they were all removed and False otherwise. 
    """
    print(f"removing apps {app_ids_to_remove}")
    return True
   
    


In [ ]:
# %%capture ouput
# result = await destructive_operations_agent.run("can you delete apps with ids [UUID('00592a68-2700-4b69-abb4-340f631fa295'), UUID('e907cca5-e928-441f-8dc6-1343d27e3a44')]")
# print(result.data)

In [ ]:
result = await destructive_operations_agent.run("delete all red Stickies")
result.data

In [ ]:
%%capture output
result_2 = await destructive_operations_agent.run("Yes, please delete them", message_history=result.new_messages())
result_2.data

In [ ]:
output.stdout

In [ ]:
pip install fabric2

In [ ]:
from fabric import Connection

In [ ]:
result = Connection('banane').run('uname', hide=True)
result.stdout

In [ ]:
ps3.s3_comm.format_public_url('asset_id')

In [ ]:
app_id =     UUID('e12be20a-b1c2-4dfa-ae69-7bd2bf02c508')
asset_id = sb.state.assetid
ps3.s3_comm.format_public_url(str(asset_id))


In [ ]:


server_name = 'banane'
import requests
from fabric import Connection
import re


def get_chunks(app_id):
    if sb.__class__.__name__ != "PDFViewer":
        return None
    sb = b.smartbits[UUID(app_id)]
    asset_id = sb.state.assetid
    asset_public_url = ps3.s3_comm.format_public_url(asset_id)
    response = requests.get(asset_public_url)
    with open(f"/tmp/{asset_id}.pdf", 'wb') as f:
        f.write(response.content)

    conn = Connection(server_name)
    conn.put(f"/tmp/{asset_id}.pdf", f'/tmp/{asset_id}.pdf')
    print("converting to md")
    conn.run(f'source /home/mahdi/miniconda3/etc/profile.d/conda.sh && conda activate /home/mahdi/mambaforge/envs/new_env && marker_single /tmp/{asset_id}.pdf /tmp/{asset_id}', shell='/bin/bash -l')
    print("Done converting... transferring data")
    conn.get(f"/tmp/{asset_id}/{asset_id}.md", f"/tmp/{asset_id}.md")

    text = open(f"/tmp/{asset_id}.md").read()
    sections = re.split(r'^\s*#{1,6}\s+', text, flags=re.MULTILINE)
    sections = [s.strip() for s in sections if s.strip()]
    return sections        


In [ ]:
import re
from groq import Groq

client = Groq()
system_prompt = """Split the text provided into a Python list that structures the input text so it can be processed by Python. If the input is a text paragraph, return a Python list where each element contains a string combining consecutive, semantically related sentences. Please make sure you don't just split over sentences -- combine those that are relevant or related to eachother. Also, include headers as part of the first sentence (seperated from the sentence by a colon), rather than giving them their own entry in the list.
For metadata, i.e., if the provided text contains title, authors, keywords, etc., only return a list where each element is a string that groups similar things, e.g., a string of authors, string of affiliations, and string of keywords. Return an empty list if the text provided consists of only references. To avoid any issues, return only a single Python list and no additional text. It's okay to add/remove spaces or remove new lines as needed to normalize the data. Please return the list in a ```python ``` block."""
def split_sections(section):
    completion = client.chat.completions.create(
        model="deepseek-r1-distill-llama-70b",
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": section
            }
        ],
        temperature=0.4,
        max_completion_tokens=4096,
        top_p=0.95,
        stream=False,
        stop=None,
    )

    pattern = re.compile(r"```python\n(.*?)```", re.DOTALL)
    match = pattern.search(completion.choices[0].message.content)
    
    if match:
        return eval(match.groups()[0])
    else:
        print("No Python code block found.")
        return none
    

In [ ]:
import time
def get_all_sentence(sections):
    i = 0
    sentences = []
    time.sleep(1)
    for section in sections:
        sentences += split_sections(section)
        i+=1
        print(f"Completed {i}", end="\t")        
    return sentences      
    

In [ ]:
sentences = get_all_sentence(sections[0:5])

In [ ]:
sentences

In [ ]:
#!pip install chromadb

In [1]:
import os
from openai import OpenAI
import chromadb
from chromadb.utils import embedding_functions
from typing import List, Dict

class DocumentEmbedder:
    def __init__(self, persist_directory: str = "./chroma_db"):
        # Remove the comma after OpenAI()
        self.client = OpenAI()  # Fixed: removed trailing comma
        self.chroma_client = chromadb.PersistentClient(path=persist_directory)
        
        # Make sure you have a valid OpenAI API key
        self.embedding_function = embedding_functions.OpenAIEmbeddingFunction(
            api_key=os.environ["OPENAI_API_KEY"],
            model_name="text-embedding-3-small"
        )

    def get_or_create_collection(self, collection_name: str):
        """Get existing collection or create a new one with the OpenAI embedding function."""
        try:
            # Try to get existing collection
            return self.chroma_client.get_collection(
                name=collection_name,
                embedding_function=self.embedding_function
            )
        except Exception as e:
            if "does not exist" in str(e):
                # Collection doesn't exist, create new one
                return self.chroma_client.create_collection(
                    name=collection_name,
                    embedding_function=self.embedding_function
                )
            else:
                # If it's some other error, raise it
                raise e

    def get_asset_ids(self, collection_name: str) -> List[str]:
        """Get all unique asset_ids in the collection."""
        collection = self.get_or_create_collection(collection_name)
        
        # Get all metadata from the collection
        result = collection.get()
        
        # Extract unique asset_ids from metadata
        asset_ids = set()
        if result['metadatas']:  # Check if there are any metadatas
            for metadata in result['metadatas']:
                asset_ids.add(metadata['asset_id'])
        
        return list(asset_ids)
    
    def process_documents(
        self,
        sentences: List[str],
        sections: List[str],
        document_title: str,
        asset_id: str,
        collection_name: str
    ): 
        """
        Process a list of sentences and store them in Chroma with metadata.
        
        Args:
            sentences: List of sentences to embed
            sections: List of section names corresponding to each sentence
            document_title: Title of the document
            asset_id: Unique identifier for the document
            collection_name: Name of the Chroma collection to use
        """
        # Validate input lengths
        if len(sentences) != len(sections):
            raise ValueError("Number of sentences must match number of sections")

        # Get or create collection
        collection = self.get_or_create_collection(collection_name)

        # Generate unique IDs for each sentence
        ids = [f"{asset_id}_sent_{i}" for i in range(len(sentences))]

        # Create metadata for each sentence
        metadatas = [
            {
                "section": section,
                "document_title": document_title,
                "asset_id": asset_id
            }
            for section in sections
        ]

        # Add documents to collection
        collection.add(
            documents=sentences,
            metadatas=metadatas,
            ids=ids
        )

        return collection

In [ ]:
embedder = DocumentEmbedder()

# Example data
sentences = [
    "This is the first sentence from the introduction.",
    "This is a sentence from the methodology section.",
    "This is a conclusion sentence."
]

sections = [
    "Introduction",
    "Methodology",
    "Conclusion"
]

# Process documents
collection = embedder.process_documents(
    sentences=sentences,
    sections=sections,
    document_title="Test Document 2",
    asset_id="doc_001",
    collection_name="my_documents"
)

In [5]:
embedder = DocumentEmbedder()

In [6]:
embedder.get_asset_ids("my_documents")

['doc_002', 'doc_001']